# AFAP Solvency Engine Validation

## Purpose of This Notebook

This notebook validates the **AFAP Phase 3 Solvency Engine**, which evaluates a company’s
capital structure and debt-servicing capacity using deterministic ratio logic.

The objectives of this notebook are to:

1. Verify that required solvency ratios are produced by the Ratio Engine Core
2. Confirm correct flag triggering and severity classification
3. Ensure schema compliance and orchestrator readiness
4. Provide an auditable, human-readable validation artifact

This notebook **does not modify engine logic**.  
It exists solely to validate correctness and integration readiness.


## AFAP Architectural Context

Within AFAP Phase 3:

- `ratio_engine_core` computes standardized financial ratios
- `solvency_engine` evaluates leverage and coverage risk
- `schema_validator` enforces output contracts
- The **orchestrator** aggregates engine outputs
- The **LLM layer** interprets already-validated results

This notebook simulates how the orchestrator will invoke the Solvency Engine
in a production pipeline.


In [1]:
# Environment setup and imports

import pandas as pd
import sys
import os

# Allow notebook to import AFAP engines
sys.path.append(os.path.abspath('..'))

from engines.ratio_engine_core import ratio_engine
from engines.solvency_engine import solvency_engine


## Load Clean Financial Statement Data

AFAP engines operate exclusively on cleaned, normalized financial statements.

This dataset has already passed:
- Structural normalization
- Line-item harmonization
- Missing value handling


In [2]:
financials = pd.read_csv("../data/cleaned/Kenya_Airways.csv")
financials.head()


,Company,Year,FS Category,FS Subcategory,Amount
0,Kenya Airways,2021,Assets,Current Assets,"25,685"
1,Kenya Airways,2021,Assets,Non-Current Assets,"129,870"
2,Kenya Airways,2021,Assets,Inventory,"2,152"
3,Kenya Airways,2021,Liabilities,Current Liabilities,"80,965"
4,Kenya Airways,2021,Liabilities,Non-Current Liabilities,"157,927"


## Run Ratio Engine Core

The Solvency Engine does **not** calculate ratios itself.
It relies on the Ratio Engine Core as a single source of truth.

This separation ensures:
- Deterministic behavior
- Cross-engine consistency
- Auditability


In [3]:
ratios_core = ratio_engine(financials)
ratios_df = pd.DataFrame(ratios_core)
ratios_df.head()


✅ ratio_engine output validated successfully.


,engine,Company,Year,metrics,flags,severity,explanation
0,ratio_engine,Kenya Airways,2021,"{'current_ratio': 0.3172358426480578, 'quick_r...",{},stable,Canonical financial ratios
1,ratio_engine,Kenya Airways,2022,"{'current_ratio': 0.3553302522298208, 'quick_r...",{},stable,Canonical financial ratios
2,ratio_engine,Kenya Airways,2023,"{'current_ratio': 0.41900024909402095, 'quick_...",{},stable,Canonical financial ratios
3,ratio_engine,Kenya Airways,2024,"{'current_ratio': 0.34834771979471035, 'quick_...",{},stable,Canonical financial ratios


## Flatten Nested Metrics for Validation

Many AFAP engines store numeric ratios under a `"metrics"` dictionary.  
To validate or use them in the Solvency Engine, we must **flatten this structure**.


In [4]:
# Flatten 'metrics' column into separate columns
metrics_df = pd.json_normalize(ratios_df['metrics'])
ratios_flat = pd.concat([ratios_df[['Company', 'Year']], metrics_df], axis=1)

ratios_flat.head()


,Company,Year,current_ratio,quick_ratio,gross_margin,operating_margin,net_margin,debt_equity,interest_coverage,asset_turnover,roa,roe
0,Kenya Airways,2021,0.317236,0.290656,-0.096880,-1.193760,-1.325302,-2.866578,-8.930116,0.451422,-0.598271,1.116719
1,Kenya Airways,2022,0.355330,0.335948,-0.048088,-1.096175,-1.377388,-2.562698,-3.891983,0.691407,-0.952336,1.488214
2,Kenya Airways,2023,0.419000,0.396477,0.057783,-0.884434,-1.071509,-2.302347,-4.704193,1.010662,-1.082933,1.385310
3,Kenya Airways,2024,0.348348,0.323013,0.323234,0.196541,0.136815,-2.514609,3.318732,1.052433,0.143989,-0.218087


## Validate Required Solvency Metrics

The Solvency Engine requires:

- `debt_equity`  
- `interest_coverage`

Optional metrics for sanity checking:

- `roa`  
- `roe`


In [5]:
for col in ["debt_equity", "interest_coverage", "roa", "roe"]:
    print(col, "min:", ratios_flat[col].min(), "max:", ratios_flat[col].max())


debt_equity min: -2.866577870573695 max: -2.3023474790494194
interest_coverage min: -8.930116118035581 max: 3.318731523783929
roa min: -1.0829327399455306 max: 0.1439889673039128
roe min: -0.21808695063889524 max: 1.4882135257655658


## Run Solvency Engine

The Solvency Engine evaluates **capital structure risk** using locked thresholds:

- High leverage → `debt_equity > 1.5`  
- Weak coverage → `interest_coverage < 1.5`  

Severity rules:

- `stable` → no flags  
- `watch` → one flag  
- `action` → both flags

Schema validation is executed **inside the engine**.


In [6]:
solvency_results = solvency_engine(ratios_flat)


✅ solvency_engine output validated successfully.


## Review Solvency Engine Output

We generate a **human-readable table** to verify:

- Severity labeling  
- Explanation text  
- One record per company-year


In [7]:
pd.DataFrame([
    {
        "Company": r["Company"],
        "Year": r["Year"],
        "Severity": r["severity"],
        "Explanation": r["explanation"]
    }
    for r in solvency_results
])


,Company,Year,Severity,Explanation
0,Kenya Airways,2021,watch,Capital structure shows solvency risk.
1,Kenya Airways,2022,watch,Capital structure shows solvency risk.
2,Kenya Airways,2023,watch,Capital structure shows solvency risk.
3,Kenya Airways,2024,stable,Solvency position acceptable.


## Output Contract Validation

Each Solvency Engine record conforms to the AFAP engine schema:

- One record per company-year  
- `metrics` dictionary with numeric ratios  
- Boolean flags for risk indicators  
- Deterministic severity label  
- Plain-language explanation

This structure allows:

- Seamless ingestion by the **AFAP orchestrator**  
- Safe downstream **LLM interpretation**


## Validation Conclusion

✔ Solvency metrics are correctly sourced from the Ratio Engine  
✔ Risk flags trigger deterministically  
✔ Severity labels behave as designed  
✔ Schema validation passes inside the engine  
✔ Engine output is orchestration-ready and AI-safe  

The AFAP Solvency Engine is **locked, auditable, and production-ready**.
